***
# Perceiver Solo Piano Maker (Version 2022.12.17)

https://github.com/asigalov61/tegridy-tools

https://github.com/lucidrains/perceiver-ar-pytorch

## Project Los Angeles
## Tegridy Code 2022

***

# GPU check

In [1]:
!nvidia-smi
# !export CUDA_VISIBLE_DEVICES=0

/bin/bash: /home/astais/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Tue Nov 22 13:07:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:02:00.0 Off |                  N/A |
| 25%   31C    P8     7W / 180W |      8MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------

# Install dependencies and import modules

In [2]:
!cd '/home/astais/Perceiver-Music-Transformer/Training-Code'
# !git clone https://github.com/asigalov61/tegridy-tools

/bin/bash: /home/astais/miniconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: line 0: cd: /home/astais/Perceiver-Music-Transformer/Training-Code: No such file or directory


In [3]:
# !pip install torch torch-summary einops sklearn matplotlib tqdm
# If it fails due to no space left on device use prefix
# TMPDIR=/var/tmp 

In [ ]:
# If the above don't work
# !pip install torch==1.12.1+cu102 torchvision==0.13.1+cu102 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu102
# !pip3 install -U scikit-learn scipy matplotlib

In [10]:
# Load modules and make data dir

print('Loading modules...')

import pickle,os,random,secrets,datetime
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)

import torch
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary
from sklearn import metrics

%cd /home/astais/perceiver-ar/Training-Code/tegridy-tools/tegridy-tools/

import TMIDIX

%cd /home/astais/perceiver-ar/Training-Code/tegridy-tools/tegridy-tools/Perceiver-AR/

from perceiver_ar_pytorch import PerceiverAR
from autoregressive_wrapper import AutoregressiveWrapper

%cd /home/astais/perceiver-ar/Training-Code/

print('Done!')

Loading modules...
/home/astais/perceiver-ar/Training-Code/tegridy-tools/tegridy-tools
/home/astais/perceiver-ar/Training-Code/tegridy-tools/tegridy-tools/Perceiver-AR
/home/astais/perceiver-ar/Training-Code
Done!


## Set Training Dataset and Directory Paths

In [13]:
# Change according to training dataset and basic load,save dir
dataset_name='maestro-v3.0.0'
basic_dir="/data/share/share1/users/astais/"

current_time = datetime.datetime.now().strftime('%Y%m%d')
preprocess_addr = basic_dir+"Unprocessed-Datasets/"+dataset_name
dataset_addr=basic_dir+"Processed-Datasets/perceiver-ar/perceiver-ar_"+dataset_name
base_save_dir=basic_dir+'Saved-Models/perceiver-ar/'
train_out_dir=basic_dir+'Training-Ouputs/perceiver-ar/'+'perceiver-ar_'+dataset_name+'_'+current_time+'/'

for addr in [dataset_addr,train_out_dir]:
    if not os.path.exists(addr):
        os.makedirs(addr)
        
print("Done!")

Done!


# (Preprocess Training Data)

In [ ]:
# @title Process MIDIs with TMIDIX MIDI Processor

sorted_or_random_file_loading_order = False # Sorted order is NOT usually recommended
dataset_ratio = 1 # Change this if you need more data

print('TMIDIX MIDI Processor')
print('Starting up...')
###########
files_count = 0
gfiles = []
train_data1 = []
###########

print('Loading MIDI files...')
print('This may take a while on a large dataset in particular.')

# os.chdir(dataset_addr)
filez = list()
for (dirpath, dirnames, filenames) in os.walk(preprocess_addr):
    filez += [os.path.join(dirpath, file) for file in filenames]
print('=' * 70)

if filez == []:
    print('Could not find any MIDI files. Please check Dataset dir...')
    print('=' * 70)
if sorted_or_random_file_loading_order:
    print('Sorting files...')
    filez.sort()
    print('Done!')
    print('=' * 70)
else:
    print('Randomizing file list...')
    random.shuffle(filez)

print('Processing MIDI files. Please wait...')
for f in tqdm(filez[:int(len(filez) * dataset_ratio)]):
    try:
        fn = os.path.basename(f)
        fn1 = fn.split('.')[0]
        #print('Loading MIDI file...')
        score = TMIDIX.midi2ms_score(open(f, 'rb').read())
        events_matrix = []
        itrack = 1
        while itrack < len(score):
            for event in score[itrack]:         
                if event[0] == 'note' and event[3] != 9:
                    events_matrix.append(event)
            itrack += 1
        
        if len(events_matrix) > 0:

          # Sorting...
          events_matrix.sort(key=lambda x: x[4], reverse=True)
          events_matrix.sort(key=lambda x: x[1])
          # recalculating timings
          for e in events_matrix:
              e[1] = int(e[1] / 10)
              e[2] = int(e[2] / 20) 
          # final processing...
          train_data1.extend([126+0, 126+128, 0+256, 0+384]) # Intro/Zero seq
          pe = events_matrix[0]
     
          for e in events_matrix:
              time = max(0, min(126, e[1]-pe[1]))
              dur = max(1, min(126, e[2]))
              ptc = max(1, min(126, e[4]))
              vel = max(1, min(126, e[5]))
              train_data1.extend([time+0, dur+128, ptc+256, vel+384])
              pe = e

          files_count += 1
        
    except KeyboardInterrupt:
        print('Quitting...')
        break  
    except:
        print('Bad MIDI:', f)
        continue

print('=' * 70)
TMIDIX.Tegridy_Any_Pickle_File_Writer(train_data1, dataset_addr+dataset_name+'/perceiver-ar_'+dataset_name)        

print('Done!')   
print('=' * 70)

# (Download and unzip training data)

In [ ]:
# # Perceiver Solo Piano Training Data Pack
# %cd /data/ironman/data1/users/astais/Processed-Datasets/
# # !wget --no-check-certificate -O 'Perceiver-Piano-Training-Data.zip' "https://onedrive.live.com/download?cid=8A0D502FC99C608F&resid=8A0D502FC99C608F%2118740&authkey=ANEK-9WanNFyalw"
# !unzip 'Perceiver-Piano-Training-Data.zip'
# # !rm 'Perceiver-Piano-Training-Data.zip'
# %cd /home/astais/Perceiver-Music-Transformer/Training-Code/

# Load Training Data

In [ ]:
# Load training data
filez = list()
for (dirpath, dirnames, filenames) in os.walk(dataset_addr):
    filez += [os.path.join(dirpath, file) for file in filenames]
print('=' * 70)
filez.sort()
print('Loading training data... Please wait...')
train_data = torch.Tensor()

for f in filez:
    train_data = torch.cat((train_data, torch.Tensor(pickle.load(open(f, 'rb')))))
    print('Loaded file:', f)
    
print('Done!')

In [ ]:
len(train_data)
print(len(train_data)//(4096 * 4))

In [ ]:
train_data[:15], train_data[-15:]

# Initialize the Model

In [ ]:
torch.cuda.empty_cache()
# Setup model

# constants
SEQ_LEN = 4096 * 4 # Total of 16k
PREFIX_SEQ_LEN = (4096 * 4) - 1024 # 15.3k
BATCH_SIZE = 1  # initial 4
NUM_BATCHES = 20*len(train_data) // SEQ_LEN // BATCH_SIZE # change multiply ratio based on dataset dimension
GRADIENT_ACCUMULATE_EVERY = 4
LEARNING_RATE = 2e-5 # initial 2e-4
VALIDATE_EVERY  = 100 
GENERATE_EVERY  = 200 
GENERATE_LENGTH = 32 

# helpers
def cycle(loader):
    while True:
        for data in loader:
            yield data

# instantiate model
model = PerceiverAR(
    num_tokens = 512, # feizc 256
    dim = 1024, # feizc 512
    depth = 24, # initial 24, feizc 8
    heads = 16, # original 16, feizc 8
    dim_head = 64,
    cross_attn_dropout = 0.5,
    max_seq_len = SEQ_LEN,
    cross_attn_seq_len = PREFIX_SEQ_LEN
)

model = AutoregressiveWrapper(model)
model.cuda()
print('Done!')
      
summary(model)

# prepare enwik8 data
class MusicDataset(Dataset):
    def __init__(self, data, seq_len):
        super().__init__()
        self.data = data
        self.seq_len = seq_len

    def __getitem__(self, index): 
        # random sampling
        # idx = secrets.randbelow((self.data.size(0) // (self.seq_len))-1) * (self.seq_len)
        
        # consequtive sampling seems to be better at 64k seq_len
        idx = index * self.seq_len
        full_seq = self.data[idx: idx + self.seq_len + 1].long()
        return full_seq.cuda()

    def __len__(self):
        return (self.data.size(0) // self.seq_len)-1

train_dataset = MusicDataset(train_data, SEQ_LEN)
val_dataset   = MusicDataset(train_data, SEQ_LEN)
train_loader  = cycle(DataLoader(train_dataset, batch_size = BATCH_SIZE))
val_loader    = cycle(DataLoader(val_dataset, batch_size = BATCH_SIZE))

# optimizer
optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training

In [ ]:
# Train the model
train_losses = []
val_losses = []
train_accs = []
val_accs = []

# Variables for saving best evalaution loss,acc
best_val_acc        = 0.0
best_val_acc_step  = -1
best_val_loss       = float("inf")
best_val_loss_step = -1
# best_acc_file = '/data/data1/users/astais/Saved-Models/perceiver-ar/'+dataset_name+'_best-val-acc.pth'
# best_loss_file = '/data/data1/users/astais/Saved-Models/perceiver-ar/'+dataset_name+'_best-val-loss.pth'

# Training Loop
for i in tqdm(range(NUM_BATCHES), mininterval=10., desc='Training'):
    new_best = False
    model.train()

    for __ in range(GRADIENT_ACCUMULATE_EVERY):
        loss, acc = model(next(train_loader))
        loss.backward()
    
    train_losses.append(loss.item())
    train_accs.append(acc.item())
    
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optim.step()
    optim.zero_grad()
       
    if i % VALIDATE_EVERY == 0:
        model.eval()
        with torch.no_grad():
            val_loss, val_acc = model(next(val_loader))
#             print(f'Validation loss: {val_loss.item()}')
#             print(f'Validation acc: {val_acc.item()}')
            val_losses.append(val_loss.item())
            val_accs.append(val_acc.item())

  
    if i % GENERATE_EVERY == 0:
        model.eval()
        inp = random.choice(val_dataset)[:-1]
        sample = model.generate(inp[None, ...], GENERATE_LENGTH)
    
    # Save Model & Plot Graphs: Trained for necessery batches and new evaluation loss or accuracy best
    if i % VALIDATE_EVERY == 0 and i>NUM_BATCHES//40  and (val_acc > best_val_acc or val_loss < best_val_loss) and (val_acc>0.8 or val_loss<0.2):

        # Check if val_acc or val_loss are best so far
        if(val_acc > best_val_acc and val_acc>0.8):
                best_val_acc = val_acc
                best_val_acc_step  = i
                best_acc_file = base_save_dir+'perceiver-ar_'+dataset_name+'_'+ str(i) + '_steps_' + str(round(float(val_accs[-1]), 4)) + '_acc.pth'
                torch.save(model.state_dict(), best_acc_file)

        if(val_loss < best_val_loss and val_loss<0.2):
                best_val_loss       = val_loss
                best_val_loss_step = i
                best_loss_file = base_save_dir+'perceiver-ar_'+dataset_name+'_'+ str(i) + '_steps_' + str(round(float(val_losses[-1]), 4)) + '_loss.pth'
                torch.save(model.state_dict(), best_loss_file)
        print('Saved model progress.')

        print("Best val acc step:", best_val_acc_step)
        print("Best val acc:", best_val_acc)
        print("Best val loss step:", best_val_loss_step)
        print("Best val loss:", best_val_loss)
        print('Done!')
        
        # Plotting Training, Evaluation Graphs
        fig, ax = plt.subplots(dpi=300)
        ax.plot([i for i in range(len(train_losses))] ,train_losses, '#607B8B')
        ax.set(xlabel='Steps', ylabel ='Training Loss', title='Perceiver-AR '+dataset_name+': Training Loss')
        ax.xaxis.set_minor_locator(AutoMinorLocator(5))
        ax.yaxis.set_minor_locator(AutoMinorLocator(5))
        ax.grid(which='minor', alpha=0.2)
        ax.grid(which='major', alpha=0.5)
        fig.savefig(train_out_dir+'perceiver-ar_'+dataset_name+'_train-loss_'+str(i) + '_steps.png')
        plt.close()

        # Training acc graph
        fig, ax = plt.subplots(dpi=300)
        ax.plot([i for i in range(len(train_accs))] ,train_accs, '#607B8B')
        ax.set(xlabel='Steps', ylabel ='Training Accuracy', title='Perceiver-AR '+dataset_name+': Training Accuracy')
        ax.xaxis.set_minor_locator(AutoMinorLocator(5))
        ax.yaxis.set_minor_locator(AutoMinorLocator(5))
        ax.grid(which='minor', alpha=0.2)
        ax.grid(which='major', alpha=0.5)
        fig.savefig(train_out_dir+'perceiver-ar_'+dataset_name+'_train-acc_'+str(i) + '_steps.png')
        plt.close()

        # Validation loss graph
        fig, ax = plt.subplots(dpi=300)
        ax.plot([100*i for i in range(len(val_losses))] ,val_losses, '#5D478B')
        ax.set(xlabel='Steps', ylabel ='Validation Loss', title='Perceiver-AR '+dataset_name+': Validation Loss')
        ax.xaxis.set_minor_locator(AutoMinorLocator(5))
        ax.yaxis.set_minor_locator(AutoMinorLocator(5))
        ax.grid(which='minor', alpha=0.2)
        ax.grid(which='major', alpha=0.5)
        fig.savefig(train_out_dir+'perceiver-ar_'+dataset_name+'_val-loss_'+str(i) + '_steps.png')
        plt.close()

        # Validation acc graph
        fig, ax = plt.subplots(dpi=300)
        plt.plot([100*i for i in range(len(val_accs))] ,val_accs, '#5D478B')
        ax.set(xlabel='Steps', ylabel ='Validation Accuracy', title='Perceiver-AR '+dataset_name+': Validation Accuracy')
        ax.xaxis.set_minor_locator(AutoMinorLocator(5))
        ax.yaxis.set_minor_locator(AutoMinorLocator(5))
        ax.grid(which='minor', alpha=0.2)
        ax.grid(which='major', alpha=0.5)
        fig.savefig(train_out_dir+'perceiver-ar_'+dataset_name+'_val-acc_'+str(i) + '_steps.png')
        plt.close()

        # Training-Validation Loss Graph
        fig, ax = plt.subplots(dpi=300)
        x_train=np.array([i for i in range(len(train_losses))])
        y_train=np.array(train_losses)
        ax.plot(x_train,y_train, '#607B8B')

        x_val=np.array([100*i for i in range(len(val_losses))])
        y_val=np.array(val_losses)
        ax.plot(x_val,y_val, '#5D478B')

        ax.set(xlabel='Steps', ylabel ='Loss', title='Perceiver-AR '+dataset_name+': Training-Validation Loss')
        ax.xaxis.set_minor_locator(AutoMinorLocator(5))
        ax.yaxis.set_minor_locator(AutoMinorLocator(5))
        ax.legend(['Training Loss','Validation Loss'], loc='upper right')
        ax.grid(which='minor', alpha=0.2)
        ax.grid(which='major', alpha=0.5)
        fig.savefig(train_out_dir+'perceiver-ar_'+dataset_name+'_train,val-loss_'+str(i) + '_steps.png')
        plt.show()
        plt.close()

        # Training-Validation Accuracy Graph
        fig, ax = plt.subplots(dpi=300)
        x_train=np.array([i for i in range(len(train_accs))])
        y_train=np.array(train_accs)
        ax.plot(x_train,y_train, '#607B8B')

        x_val=np.array([100*i for i in range(len(val_accs))])
        y_val=np.array(val_accs)
        ax.plot(x_val,y_val, '#5D478B')

        ax.set(xlabel='Steps', ylabel ='Accuracy', title='Perceiver-AR '+dataset_name+': Training-Validation Accuracy')
        ax.xaxis.set_minor_locator(AutoMinorLocator(5))
        ax.yaxis.set_minor_locator(AutoMinorLocator(5))
        ax.legend(['Training Accuracy','Validation Accuracy'], loc='lower right')
        ax.grid(which='minor', alpha=0.2)
        ax.grid(which='major', alpha=0.5)
        fig.savefig(train_out_dir+'perceiver-ar_'+dataset_name+'_train,val-acc_'+str(i) + '_steps.png')
        plt.show()
        plt.close()
    
    # Early Stopping: If best val_loss or acc hasn't appeared for 1000 batches stop training.
    ### Add minimum step to apply early stopping
    if(val_acc < best_val_acc and val_loss < best_val_loss and i>best_val_loss_step+1000 and i>best_val_acc_step+1000):
        print("Early Stopping: Training Finished!")
        break


# Save stats graphs and value arrays

In [ ]:
# Save training, validation value arrays
with open(train_out_dir+'perceiver-ar_'+dataset_name+"_"+current_time+'.npy', 'wb') as f:
    np.save(f, np.array(train_losses))
    np.save(f, np.array(train_accs))
    np.save(f, np.array(val_losses))
    np.save(f, np.array(val_accs))

# Training loss graph
fig, ax = plt.subplots(dpi=300)
ax.plot([i for i in range(len(train_losses))] ,train_losses, '#607B8B')
ax.set(xlabel='Steps', ylabel ='Training Loss', title='Perceiver-AR '+dataset_name+': Training Loss')
ax.xaxis.set_minor_locator(AutoMinorLocator(5))
ax.yaxis.set_minor_locator(AutoMinorLocator(5))
ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
fig.savefig(train_out_dir+'perceiver-ar_'+dataset_name+'_train-loss.png')
plt.show()
plt.close()

# Training acc graph
fig, ax = plt.subplots(dpi=300)
ax.plot([i for i in range(len(train_accs))] ,train_accs, '#607B8B')
ax.set(xlabel='Steps', ylabel ='Training Accuracy', title='Perceiver-AR '+dataset_name+': Training Accuracy')
ax.xaxis.set_minor_locator(AutoMinorLocator(5))
ax.yaxis.set_minor_locator(AutoMinorLocator(5))
ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
fig.savefig(train_out_dir+'perceiver-ar_'+dataset_name+'_train-acc.png')
plt.show()
plt.close()

# Validation loss graph
fig, ax = plt.subplots(dpi=300)
ax.plot([100*i for i in range(len(val_losses))] ,val_losses, '#5D478B')
ax.set(xlabel='Steps', ylabel ='Validation Loss', title='Perceiver-AR '+dataset_name+': Validation Loss')
ax.xaxis.set_minor_locator(AutoMinorLocator(5))
ax.yaxis.set_minor_locator(AutoMinorLocator(5))
ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
fig.savefig(train_out_dir+'perceiver-ar_'+dataset_name+'_val-loss.png')
plt.show()
plt.close()

# Validation acc graph
fig, ax = plt.subplots(dpi=300)
plt.plot([100*i for i in range(len(val_accs))] ,val_accs, '#5D478B')
ax.set(xlabel='Steps', ylabel ='Validation Accuracy', title='Perceiver-AR '+dataset_name+': Validation Accuracy')
ax.xaxis.set_minor_locator(AutoMinorLocator(5))
ax.yaxis.set_minor_locator(AutoMinorLocator(5))
ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
fig.savefig(train_out_dir+'perceiver-ar_'+dataset_name+'_val-acc.png')
plt.show()
plt.close()

# Training-Validation Loss Graph
fig, ax = plt.subplots(dpi=300)
x_train=np.array([i for i in range(len(train_losses))])
y_train=np.array(train_losses)
ax.plot(x_train,y_train, '#607B8B')

x_val=np.array([100*i for i in range(len(val_losses))])
y_val=np.array(val_losses)
ax.plot(x_val,y_val, '#5D478B')

ax.set(xlabel='Steps', ylabel ='Loss', title='Perceiver-AR '+dataset_name+': Training-Validation Loss')
ax.xaxis.set_minor_locator(AutoMinorLocator(5))
ax.yaxis.set_minor_locator(AutoMinorLocator(5))
ax.legend(['Training Loss','Validation Loss'], loc='upper right')
ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
fig.savefig(train_out_dir+'perceiver-ar_'+dataset_name+'_train,val-loss.png')
plt.show()
plt.close()

# Training-Validation Accuracy Graph
fig, ax = plt.subplots(dpi=300)
x_train=np.array([i for i in range(len(train_accs))])
y_train=np.array(train_accs)
ax.plot(x_train,y_train, '#607B8B')

x_val=np.array([100*i for i in range(len(val_accs))])
y_val=np.array(val_accs)
ax.plot(x_val,y_val, '#5D478B')

ax.set(xlabel='Steps', ylabel ='Accuracy', title='Perceiver-AR '+dataset_name+': Training-Validation Accuracy')
ax.xaxis.set_minor_locator(AutoMinorLocator(5))
ax.yaxis.set_minor_locator(AutoMinorLocator(5))
ax.legend(['Training Accuracy','Validation Accuracy'], loc='lower right')
ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)
fig.savefig(train_out_dir+'perceiver-ar_'+dataset_name+'_train,val-acc.png')
plt.show()
plt.close()


# Load/Reload the Model

In [ ]:
# Load model
fname=base_save_dir+"perceiver-ar_ailabs1k17_10300_steps_0.0069_loss.pth"
# constants
SEQ_LEN = 4096 * 4 # Total of 16k
PREFIX_SEQ_LEN = (4096 * 4) - 1024 # 15.3k

model = PerceiverAR(
    num_tokens = 512,
    dim = 1024,
    depth = 24, #24,
    heads = 16, 
    dim_head = 64,
    cross_attn_dropout = 0.5,
    max_seq_len = SEQ_LEN,
    cross_attn_seq_len = PREFIX_SEQ_LEN
)
model = AutoregressiveWrapper(model)
model.cuda()

state_dict = torch.load(fname)

model.load_state_dict(state_dict)

model.eval()

# Model stats

summary(model)

In [ ]:
# Plot Token Embeddings

cos_sim = metrics.pairwise.cosine_similarity(
    model.net.token_emb.weight.cpu().detach().numpy()[0].reshape(-1, 1)
)
plt.figure(figsize=(8, 8))
plt.imshow(cos_sim, cmap="inferno", interpolation="none")
im_ratio = cos_sim.shape[0] / cos_sim.shape[1]
plt.colorbar(fraction=0.046 * im_ratio, pad=0.04)
plt.xlabel("Position")
plt.ylabel("Position")
plt.tight_layout()
plt.plot()
plt.savefig(train_out_dir+"perceiver-ar_"+dataset_name+"_positional-embeddings.png", bbox_inches="tight")

# EVAL

In [ ]:
# Custom MIDI option

print('Loading custom MIDI file...')

f = '/home/astais/Perceiver-Music-Transformer/Training-Code/tegridy-tools/tegridy-tools/seed2.mid'

score = TMIDIX.midi2ms_score(open(f, 'rb').read())

events_matrix = []

itrack = 1

while itrack < len(score):
    for event in score[itrack]:         
        if event[0] == 'note' and event[3] != 9:
            events_matrix.append(event)
    itrack += 1

if len(events_matrix) > 0:

    # Sorting...
    events_matrix.sort(key=lambda x: x[4], reverse=True)
    events_matrix.sort(key=lambda x: x[1])

    # recalculating timings
    for e in events_matrix:
        e[1] = int(e[1] / 10)
        e[2] = int(e[2] / 20)

    # final processing...
    inputs = []
    
    inputs.extend([126+0, 126+128, 0+256, 0+384]) # Intro/Zero sequence

    pe = events_matrix[0]
    for e in events_matrix:

        time = max(0, min(126, e[1]-pe[1]))
        dur = max(1, min(126, e[2]))

        ptc = max(1, min(126, e[4]))
        vel = max(1, min(126, e[5]))

        inputs.extend([time+0, dur+128, ptc+256, vel+384])

        pe = e
        
print('Done!')

In [ ]:
# Generate

import time

model.eval()
inp = val_dataset[0]

print(inp)

start_time = time.time()

out = model.generate(inp[None, ...], 
                     128, 
                     temperature=0.6)

print(time.time() - start_time, "seconds")
print(out)

out1 = out.cpu().tolist()[0]

In [ ]:
# Convert to MIDI
if len(out1) != 0:
    
    song = out1
    song_f = []
    time = 0
    dur = 0
    vel = 0
    pitch = 0
    channel = 0

    son = []

    song1 = []

    for s in song:
      if s > 127:
        son.append(s)

      else:
        if len(son) == 4:
          song1.append(son)
        son = []
        son.append(s)
    
    for s in song1:

        channel = 0 # Piano

        time += s[0] * 10
            
        dur = (s[1]-128) * 20
        
        pitch = (s[2]-256)

        vel = (s[3]-384)
                                  
        song_f.append(['note', time, dur, channel, pitch, vel ])

    detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                        output_signature = 'Perceiver',  
                                                        output_file_name = '/data/ironman/data1/users/astais/Midi-Outputs/perceiver-ar/perceiver-ar_'+dataset_name+'_'+current_time, 
                                                        track_name='Project Los Angeles',
                                                        list_of_MIDI_patches=[0, 24, 32, 40, 42, 46, 56, 71, 73, 0, 53, 19, 0, 0, 0, 0],
                                                        number_of_ticks_per_quarter=500)

    print('Done!')